In [5]:
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup
import os
import json
from tqdm import tqdm
import re
import time

BASE_URL = "https://www.flashscore.com"

# Adjusted scrolling mechanism to scroll until no more matches are loaded
async def get_match_id_list(tournament, league):
    url = f"{BASE_URL}/tennis/{league}/{tournament}/results/"
    browser = await launch(headless=True)
    page = await browser.newPage()
    await page.setUserAgent("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    try:
        await page.goto(url, {"waitUntil": "networkidle2"})
        
        last_height = await page.evaluate('document.body.scrollHeight')

        while True:
            await page.evaluate('window.scrollBy(0, window.innerHeight)')
            await asyncio.sleep(1)
            new_height = await page.evaluate('document.body.scrollHeight')
            
            if new_height == last_height:
                break
            
            last_height = new_height
        
        content = await page.content()
        soup = BeautifulSoup(content, "html.parser")
        all_matches = soup.select(".event__match.event__match--static.event__match--twoLine")
        
        print(f"Found {len(all_matches)} matches.")
        
        return all_matches
    
    finally:
        await browser.close()

# Example usage
async def main():
    matches = await get_match_id_list("wimbledon", "atp-singles")
    # Process matches further if needed

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())


/users/eleves-b/2021/mathias.grau/.local/lib/python3.9/site-packages/pygments/regexopt.py:77: RuntimeWarning: coroutine 'get_match_id_list' was never awaited
  '|'.join(regex_opt_inner(list(group[1]), '')


RuntimeError: This event loop is already running